# Non-negative matrix factorization with sparsity constraints using Autograd

In a [previous post](./nnmf-tensorflow.html), we had seen how to perfom non-negative matrix factorization (NNMF) using Tensorflow. In this post, we will look at performing NNMF using [Autograd](https://github.com/HIPS/autograd). Like Tensorflow, Autograd allows automatic gradient calculation.

### Customary imports

In [22]:
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Creating the matrix to be decomposed

In [23]:
A = np.array([[3, 4, 5, 2],
                   [4, 4, 3, 3],
                   [5, 5, 4, 3]], dtype=np.float32).T

### Masking one entry

In [24]:
len(A[A!=0])

12

In [25]:
np.sum(np.abs(A))

45.0

In [26]:
A[0, 0] = np.NAN

In [27]:
A.shape

(4, 3)

In [28]:
np.sum(A[A!=0])

nan

### Defining the cost function

In [29]:
H

array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799,  0.97727788],
       [ 0.95008842,  0.15135721,  0.10321885],
       [ 0.4105985 ,  0.14404357,  1.45427351],
       [ 0.76103773,  0.12167502,  0.44386323],
       [ 0.33367433,  1.49407907,  0.20515826],
       [ 0.3130677 ,  0.85409574,  2.55298982],
       [ 0.6536186 ,  0.8644362 ,  0.74216502],
       [ 2.26975462,  1.45436567,  0.04575852],
       [ 0.18718385,  1.53277921,  1.46935877]])

In [171]:
def cost(W, H, lam=1, norm="2,1"):
    pred = np.dot(W, H)
    mask = ~np.isnan(A)
    if norm=="2,1":
        # First L2 over rows and then L1 over result
        penalty = np.mean(np.sqrt(np.sum(H**2, axis=0)))
    elif norm=="0":
        penalty = np.sum(H[H!=0])
    return np.sqrt(((pred - A)[mask].flatten() ** 2).mean(axis=None)) + lam*penalty

### Decomposition params

In [172]:
rank = 10
learning_rate=0.01
n_steps = 10000

### Gradient of cost wrt params W and H

In [173]:
from autograd import grad, multigrad
grad_cost= multigrad(cost, argnums=[0,1])

### Main gradient descent routine

In [178]:
shape = A.shape
np.random.seed(0)
H =  np.abs(np.random.randn(rank, shape[1]))
W =  np.abs(np.random.randn(shape[0], rank))
print "Iteration, Cost"
for i in range(n_steps):
    
    if i%100==0:
        print "*"*20
        print i,",", cost(W, H)
        mask = ~np.isnan(A)
        print np.sqrt(((np.dot(W, H) - A)[mask].flatten() ** 2).mean(axis=None)), np.mean(np.sqrt(np.sum(H**2, axis=0)))
    del_W, del_H = grad_cost(W, H)
    W =  W-del_W*learning_rate
    H =  H-del_H*learning_rate
    
    # Ensuring that W, H remain non-negative. This is also called projected gradient descent
    W[W<0] = 0.
    H[H<0] = 0.

Iteration, Cost
********************
0 , 7.8074674545
4.11660945144 3.69085800306
********************
100 , 3.77696419877
0.831569470895 2.94539472787
********************
200 , 2.9374193837
0.309072202323 2.62834718138
********************
300 , 2.47142401099
0.060994596725 2.41042941427
********************
400 , 2.20564132862
0.00404946380567 2.20159186482
********************
500 , 2.01255024411
0.00418481657025 2.00836542754
********************
600 , 1.8598780115
0.0188892164873 1.84098879501
********************
700 , 1.7179434289
0.0144148799476 1.70352854895
********************
800 , 1.60671559017
0.0159317133041 1.59078387687
********************
900 , 1.5177987394
0.0168379626974 1.5009607767
********************
1000 , 1.44735576901
0.017713885629 1.42964188338
********************
1100 , 1.39085910342
0.0185650531293 1.37229405029
********************
1200 , 1.34472119362
0.0194076215818 1.32531357204
********************
1300 , 1.30616339893
0.0202463274503 1.2859170714

In [111]:
H[H<0] = 0.

In [179]:
pd.DataFrame(W)

,0,1,2,3,4,5,6,7,8,9
0,0.848175,1.642954,2.283753,4.442932,0.844876,0.750217,3.248048,2.581215,1.443529,1.266878
1,1.580067,1.950030,2.858395,4.329612,0.849971,0.620860,2.977379,1.801214,1.923315,0.986069
2,4.598746,2.939838,1.724074,3.188869,1.022232,0.441897,0.621784,1.059382,2.950799,1.136291
3,0.000000,0.131597,1.446627,2.732701,0.000000,2.043358,2.582847,0.967172,2.110048,0.347477


In [180]:
pd.DataFrame(H)

,0,1,2
0,0.397170,0.049224,0.111182
1,0.264946,0.073690,0.185207
2,0.167738,0.195871,0.251816
3,0.297669,0.354562,0.428839
4,0.094635,0.029068,0.091157
5,0.030001,0.178831,0.035849
6,0.071021,0.296880,0.300762
7,0.103100,0.145153,0.228574
8,0.251819,0.221511,0.132180
9,0.103313,0.064615,0.116890


In [181]:
pred = np.dot(W, H)
pred_df = pd.DataFrame(pred).round()
pred_df

,0,1,2
0,4.0,4.0,5.0
1,4.0,4.0,5.0
2,5.0,3.0,4.0
3,2.0,3.0,3.0


In [182]:
pd.DataFrame(A)

,0,1,2
0,NaN,4.0,5.0
1,4.0,4.0,5.0
2,5.0,3.0,4.0
3,2.0,3.0,3.0


In [74]:
np.count_nonzero(H)

15

In [76]:
H.size

15

In [75]:
H

array([[ 0.67465207,  0.30721673,  0.47904857],
       [ 0.18433676,  0.50632065,  0.63834613],
       [ 0.04810152,  0.28539869,  0.25933821],
       [ 0.46973566,  0.45846022,  0.44300948],
       [ 0.48211291,  0.4235707 ,  0.57590783]])